[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dennislwei/icx/blob/master/examples/mexgen/quick_start.ipynb)


General tip for colab - change the runtime type to GPU for faster run

**RUNNING COLAB NOTEBOOK FROM A PRIVATE REPO**

- Make sure you are a member of the repo.
- Go to https://colab.research.google.com/, open "Settings", and in "Github" click "Authorize with Github" and follow the steps. Make sure to enable "Access private repositories and organizations".
- Generate an access token from github:
    - Goto https://github.com/settings/tokens/new -> in the scopes select "repo" and "read:org". Generate the token and save a copy - you won't see it again after you navigate away from the page.
- Run commands in the "Install ICX360 for Colab" section

**TODO BEFORE OSS RELEASE:**
- change google colab url, change any RITS references

# MExGen Quick Start

This notebook shows a simple example of using MExGen (Multi-Level Explanations for Generative Language Models) to get users started. For more complete examples, please see the notebooks on [question answering](https://github.com/dennislwei/icx/blob/master/examples/mexgen/question_answering.ipynb), [summarization](https://github.com/dennislwei/icx/blob/master/examples/mexgen/summarization.ipynb), and [RAG](https://github.com/dennislwei/icx/blob/master/examples/mexgen/RAG.ipynb).

### Install ICX360 for Colab (for private repo) - skip if you have icx360 installed:

In [1]:
# Get the github token securely
from getpass import getpass
token = getpass("Enter your GitHub Personal Access Token: ")

# Clone the private
!sudo apt install gh
!echo "$token" | gh auth login --hostname github.com --with-token
!gh auth status | echo
!rm -rf icx
!git clone https://$token@github.com/dennislwei/icx.git

# This installs the package anyway. Need to fix this issue.
%cd "icx"
!pip install uv
!uv pip install .
%cd ..

Streaming output truncated to the last 5000 lines.
nvidia-curand-cu12         ------------------------------ 1.12 MiB/53.70 MiB
nvidia-cusolver-cu12       ------------------------------ 1.21 MiB/122.01 MiB
nvidia-cusparse-cu12       ------------------------------ 1.11 MiB/197.84 MiB
nvidia-cufft-cu12          ------------------------------ 1.41 MiB/201.66 MiB
nvidia-cublas-cu12         ------------------------------ 1.14 MiB/346.60 MiB
en-core-web-trf            ------------------------------ 9.47 MiB/436.23 MiB
   Building icx360 @ file:///content/icx
      Built detect-secrets @ git+https://github.com/ibm/detect-secrets.git@cb803c6f7
⠴ Preparing packages... (26/48)
bert-score                 ------------------------------ 48.00 KiB/59.70 KiB
jupyter-client             ------------------------------ 123.50 KiB/130.36 KiB
jupyter-server             ------------------------------ 155.98 KiB/377.84 KiB
nvidia-cuda-runtime-cu12   ------------------------------ 489.50 KiB/863.02 KiB
jedi  

In [2]:
# %%bash

# UNAME="dennislwei"
# FOLDER="icx"
# REPO="github.com/${UNAME}/${FOLDER}"
# PACKAGE="icx360"
# echo $REPO

# sudo apt install gh
# echo "$token" | gh auth login --hostname github.com --with-token
# rm -rf $FOLDER
# git clone https://$token@github.com/$UNAME/${FOLDER}.git

In [3]:
# !https://$token@github.com/$UNAME/${FOLDER}.git

In [4]:
# %%bash
# UNAME="dennislwei"
# FOLDER="icx"
# REPO="github.com/${UNAME}/${FOLDER}"
# PACKAGE="icx360"
# echo $REPO

# if python -c "import $PACKAGE" &> /dev/null; then
#     echo "Package '$PACKAGE' is already installed."
# else
#     echo "Package '$PACKAGE' is not installed. Cleaning existing folder and downloading ..."
#     sudo apt install gh
#     rm -rf $FOLDER

#     # Clone the repo with error handling
#     if gh repo clone $REPO; then
#         echo "Repository cloned successfully."
#     else
#         echo "Error: Failed to clone the repository, reauthenticating and trying again ..."
#         echo "$token" | gh auth login --hostname github.com --with-token
#         if gh repo clone $REPO; then
#             echo "Repository cloned successfully."
#         else
#             echo "Error: Failed again, quitting ..."
#         fi
#     fi
# fi

In [5]:
# !cd icx; ls

### Import packages

Standard packages

In [6]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import torch
from transformers import BartForConditionalGeneration, BartTokenizerFast
from IPython.display import display

ICX360 classes

In [7]:
from icx360.algorithms.mexgen import CLIME    # explainer
from icx360.utils.general_utils import select_device    # set device automatically
from icx360.utils.model_wrappers import HFModel    # model wrapper

In [8]:
device = select_device()
display(device)

device(type='cuda')

### Load input

The task for this example is summarization, using a document from the Extreme Summarization (XSum) dataset. For convenience, the document has been saved to a file that we will load.

In [9]:
# with open("quick_start_doc.txt", "r") as f:
#     document = f.read()
# print(document)

document = """On Thursday, a human skull was found alongside the M54 slip road by workers doing a survey of the junction four roundabout, near Telford.
Police confirmed the skull was that of an adult male and had been there for at least two years.
West Mercia Police said "further skeletal remains" were found close to the skull.
The eastbound entry slip road remains partially closed.
Det Chief Insp Neil Jamieson said: "We are in the very early stages of this investigation and inquiries are ongoing."
He said further forensic examinations and excavations were being carried out and police had been in contact with neighbouring forces asking for information about people who had been reported missing.
Archaeological experts may be called in to help with the investigation.
"This will be a lengthy process but we will continue to update the public in due course," he added."""
print(document)

On Thursday, a human skull was found alongside the M54 slip road by workers doing a survey of the junction four roundabout, near Telford.
Police confirmed the skull was that of an adult male and had been there for at least two years.
West Mercia Police said "further skeletal remains" were found close to the skull.
The eastbound entry slip road remains partially closed.
Det Chief Insp Neil Jamieson said: "We are in the very early stages of this investigation and inquiries are ongoing."
He said further forensic examinations and excavations were being carried out and police had been in contact with neighbouring forces asking for information about people who had been reported missing.
Archaeological experts may be called in to help with the investigation.
"This will be a lengthy process but we will continue to update the public in due course," he added.


The document can also be retrieved from the dataset (and other examples can be loaded) by uncommenting the following cell:

In [10]:
# from datasets import load_dataset
# dataset = load_dataset('xsum', split='test', trust_remote_code=True)
# document = dataset["document"][88]
# print(document)

### Load model to explain

We will use a small summarization model from HuggingFace that can be run on a CPU only (although a GPU would be significantly faster).

In [11]:
model_name = "sshleifer/distilbart-xsum-12-6"
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)
tokenizer = BartTokenizerFast.from_pretrained(model_name, add_prefix_space=True)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/611M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/611M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

We wrap the model with a common API (`HFModel`) that the explainer will use.

In [12]:
wrapped_model = HFModel(model, tokenizer)

### Instantiate explainer

Below we instantiate a MExGen C-LIME explainer. This explainer attributes an importance score to each part of the input document by calling the summarization model on perturbed versions of the input. Parameters for the explainer:
- `scalarizer`: The explainer requires a "scalarizer" to quantify how different are the output summaries for perturbed inputs from the output summary for the original input. For this we use the `"prob"` scalarizer, which computes the probability of generating the original output conditioned on perturbed inputs.
- `segmenter`: The explainer will use the spaCy model `"en_core_web_sm"` to segment the document into sentences.

In [13]:
explainer = CLIME(wrapped_model, scalarizer="prob", segmenter="en_core_web_sm")

### Call explainer

We call the explainer's `explain_instance` method on the input document with default parameters. This will attribute an importance score to each sentence in the document. (This may take a minute if running on a CPU.)

In [14]:
output_dict = explainer.explain_instance(document)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


toma_get_probs batch size = 37


### Look at output

The explainer returns a dictionary. The `"output_orig"` item shows the output summary for the original document.

In [15]:
display(output_dict["output_orig"].output_text)

['Police investigating the discovery of a human skull on a motorway in Shropshire have said further skeletal remains have been found.']

The `"attributions"` item contains the sentences (`"units"`) that the document has been split into along with their importance scores (`"prob"`). These are displayed below as a pandas DataFrame.

In [16]:
df1 = pd.DataFrame(output_dict["attributions"])[["units", "prob", "unit_types"]].sort_values(
                                by='prob', ascending=False, inplace=False)
styles = [
    {'selector': '.col0', 'props': [('width', '450px'), ('font-weight', 'bold')]}, # units
    {'selector': '.col1', 'props': [('width', '50px')]}, # prob
    {'selector': '.col2', 'props': [('width', '50px')]}, # unit_types
    ]
styled = df1.style.set_table_styles(styles)

display(styled)

,units,prob,unit_types
2,"West Mercia Police said ""further skeletal remains"" were found close to the skull.",0.629328,s
0,"On Thursday, a human skull was found alongside the M54 slip road by workers doing a survey of the junction four roundabout, near Telford.",0.514589,s
4,"Det Chief Insp Neil Jamieson said: ""We are in the very early stages of this investigation and inquiries are ongoing.",0.030449,s
3,The eastbound entry slip road remains partially closed.,0.022306,s
7,"""This will be a lengthy process but we will continue to update the public in due course,"" he added.",0.009262,s
1,Police confirmed the skull was that of an adult male and had been there for at least two years.,0.008356,s
5,""" He said further forensic examinations and excavations were being carried out and police had been in contact with neighbouring forces asking for information about people who had been reported missing.",0.008152,s
6,Archaeological experts may be called in to help with the investigation.,-0.004774,s


The most important sentence suggests that the model has closely paraphased text ("further skeletal remains have been found"), while the second most important sentence suggests that it also abstracts concepts ("M54 slip road" --> "motorway").